In [1]:
from atproto import Client, AsyncClient 
from dotenv import dotenv_values 
import asyncio 
import numpy as np
import networkx as nx 
import matplotlib.pyplot as plt 
import json
import atproto_client.models as models
import re
from collections import Counter
from pyvis.network import Network
from IPython.display import display, IFrame

In [2]:
dotenv = dotenv_values('.env')

client = AsyncClient() 
await client.login(dotenv['USERNAME'], dotenv['PASSWORD']);

In [3]:
async def get_posts_by_tag(tag: str, limit=10):
    response = await client.app.bsky.feed.search_posts(
        params=models.AppBskyFeedSearchPosts.Params(
            q=tag,  
            limit=limit
        )
    )

    return response.posts

def extract_tags(posts):
    tags = []
    pattern = re.compile('\#\w+')
    
    for post in posts: 
        text = post.record.text
        [tags.append(tag) for tag in pattern.findall(text.lower())]

    return tags

async def get_tag_data(tag: str, limit=10):
    posts = await get_posts_by_tag(tag, limit)
    tags = extract_tags(posts) 

    tasks = [get_posts_by_tag(f"{t} {tag}") for t in tags]
    results = await asyncio.gather(*tasks)

    return [(tag, extract_tags(posts)) for tag, posts in zip(tags, results)]

tags = await get_tag_data("#germany", 5)
all_seen_tags = Counter() 

for tag, others in tags: 
    all_seen_tags.update(others)
all_seen_tags.update([tag for tag, _ in tags])

In [5]:
G = nx.Graph()

for tag, others in tags: 
    if all_seen_tags[tag] > 1: 
        
        G.add_edge("#germany", tag) 
    [G.add_edge(tag, other) for other in others if all_seen_tags[other] > 1]

In [11]:
import plotly.graph_objects as go

scale_factor = 10
pos = nx.spring_layout(G, k=50)  
edge_x = []
edge_y = []

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])  
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x, 
    y=edge_y,
    line=dict(width=0.2, color="#888"),
    hoverinfo="none",
    mode="lines"
)

node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_size = np.array([all_seen_tags[node] for node in G.nodes()])
max_node_size = node_size.max()
scaled = (node_size / max_node_size) * 20 + 5 
hover_text = [f"{tag}<br>Seen {all_seen_tags[tag]} times" for tag in G.nodes()]

for node in G.nodes():
    pos[node] = np.array(pos[node]) * (1 + scale_factor * (scaled[list(G.nodes()).index(node)] / 10))

# Create node traces (scatter points)
node_trace = go.Scatter(
    x=node_x, 
    y=node_y,
    mode="markers",
    text=[str(node) for node in G.nodes()],
    textposition="top center",
    hoverinfo="text",
    hovertext=hover_text,
    marker=dict(
        size=node_size, 
        color="blue",
        line=dict(width=2, color="black")
    )
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    showlegend=False,
    hovermode="closest",
    margin=dict(b=0, l=0, r=0, t=0),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    dragmode='pan',
)

fig.show()
